# Big Assignment of Parallel Processing (Class A)
## Sales Transaction Data Analysis Using Pyspark
1. Michael Effendy (F1D022012)
2. Dara Finas Elen (F1D022038)
3. Nazila Imkani (F1D022084)
4. Wahyuni Sulastri (F1D022101)
5. Fadila Ramdhani Muaz (F1D022119)

Kaggle link for the dataset: https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business

Link for another google colab (without parallel processing) for time comparison: https://colab.research.google.com/drive/1anYJgJyB-b6BosFu6QP9GR9_TImEgLjO#scrollTo=tJSkWAhxqU68

In [ ]:
import psutil
print(psutil.cpu_percent())

100.0


In [ ]:
!pip install pyspark
print(psutil.cpu_percent())

75.2


In [ ]:
from pyspark.sql import SparkSession
print(psutil.cpu_percent())

66.7


In [ ]:
spark = SparkSession.builder.appName("Sales Analysis").getOrCreate()
print(psutil.cpu_percent())

100.0


In [ ]:
import time
mulai_ukur = time.time()

In [ ]:
df = spark.read.csv("Sales Transaction v.4a.csv", header=True, inferSchema=True)
print(psutil.cpu_percent())

93.8


In [ ]:
from pyspark.sql.functions import date_format, to_date
df = df.withColumn("Date", to_date(df["Date"], "MM/dd/yy"))
print(psutil.cpu_percent())

83.3


In [ ]:
df.show(10)
print(psutil.cpu_percent())

+-------------+----------+---------+--------------------+-----+--------+----------+--------------+
|TransactionNo|      Date|ProductNo|         ProductName|Price|Quantity|CustomerNo|       Country|
+-------------+----------+---------+--------------------+-----+--------+----------+--------------+
|       581482|2019-12-09|    22485|Set Of 2 Wooden M...|21.47|      12|     17490|United Kingdom|
|       581475|2019-12-09|    22596|Christmas Star Wi...|10.65|      36|     13069|United Kingdom|
|       581475|2019-12-09|    23235|Storage Tin Vinta...|11.53|      12|     13069|United Kingdom|
|       581475|2019-12-09|    23272|Tree T-Light Hold...|10.65|      12|     13069|United Kingdom|
|       581475|2019-12-09|    23239|Set Of 4 Knick Kn...|11.94|       6|     13069|United Kingdom|
|       581475|2019-12-09|    21705|Bag 500g Swirly M...|10.65|      24|     13069|United Kingdom|
|       581475|2019-12-09|    22118|Joy Wooden Block ...|11.53|      18|     13069|United Kingdom|
|       58

In [ ]:
df.createOrReplaceTempView("sales")
print(psutil.cpu_percent())

100.0


In [ ]:
product_terlaris = spark.sql("SELECT productNo, productName, SUM(quantity) as total_terjual FROM sales GROUP BY productNo, productName ORDER BY total_terjual DESC LIMIT 10")
print("Produk terlaris dengan jumlah penjualannya" )
product_terlaris.show()
print(psutil.cpu_percent())

Produk terlaris dengan jumlah penjualannya
+---------+--------------------+-------------+
|productNo|         productName|total_terjual|
+---------+--------------------+-------------+
|    22197|      Popcorn Holder|        56450|
|    84077|World War 2 Glide...|        53847|
|   85099B|Jumbo Bag Red Ret...|        47363|
|    84879|Assorted Colour B...|        36445|
|    21212|Pack Of 72 Retros...|        36231|
|   85123A|Cream Hanging Hea...|        35378|
|    23084|  Rabbit Night Light|        30680|
|    22492|Mini Paint Set Vi...|        26437|
|    22616|Pack Of 12 London...|        25907|
|    21977|Pack Of 60 Pink P...|        24753|
+---------+--------------------+-------------+

96.9


In [ ]:
pelanggan_setia = spark.sql("SELECT customerNo, COUNT(*) as total_transaksi FROM sales GROUP BY customerNo ORDER BY total_transaksi DESC LIMIT 10")
print("Pelanggan setia dengan jumlah transaksi yang dilakukannya")
pelanggan_setia.show()
print(psutil.cpu_percent())

Pelanggan setia dengan jumlah transaksi yang dilakukannya
+----------+---------------+
|customerNo|total_transaksi|
+----------+---------------+
|     17841|           7967|
|     14911|           5800|
|     14096|           5093|
|     12748|           4627|
|     14606|           2773|
|     15311|           2464|
|     14646|           2066|
|     13089|           1888|
|     13263|           1672|
|     14298|           1640|
+----------+---------------+

96.5


In [ ]:
negara_penjualan_tertinggi = spark.sql("SELECT country, SUM(Price) as pendapatan FROM sales GROUP BY country ORDER BY pendapatan DESC LIMIT 10")
print("Negara dengan angka pendapatan penjualan tertinggi")
negara_penjualan_tertinggi.show()
print(psutil.cpu_percent())

Negara dengan angka pendapatan penjualan tertinggi
+--------------+------------------+
|       country|        pendapatan|
+--------------+------------------+
|United Kingdom| 6138222.879999699|
|       Germany| 134465.6999999991|
|        France|130757.04999999912|
|          EIRE|104928.15999999884|
|       Belgium| 33477.43000000011|
|         Spain|32597.640000000138|
|   Switzerland|30526.950000000146|
|   Netherlands|27700.860000000088|
|      Portugal| 22151.31000000004|
|     Australia|21767.810000000056|
+--------------+------------------+

96.1


In [ ]:
negara_transaksi_tertinggi = spark.sql("SELECT country, COUNT(transactionNo) as total_transaksi FROM sales GROUP BY country ORDER BY total_transaksi DESC LIMIT 10")
print("Negara dengan jumlah transaksi tertinggi")
negara_transaksi_tertinggi.show()
print(psutil.cpu_percent())

Negara dengan jumlah transaksi tertinggi
+--------------+---------------+
|       country|total_transaksi|
+--------------+---------------+
|United Kingdom|         485095|
|       Germany|          10675|
|        France|          10526|
|          EIRE|           8048|
|       Belgium|           2539|
|         Spain|           2430|
|   Switzerland|           2336|
|   Netherlands|           2330|
|      Portugal|           1848|
|     Australia|           1704|
+--------------+---------------+

99.6


In [ ]:
quantity_terbanyak = spark.sql("SELECT quantity, COUNT(*) as jumlah FROM sales GROUP BY quantity ORDER BY jumlah DESC LIMIT 10")
print("Berapa banyak item yang dibeli dalam satu transaksi")
quantity_terbanyak.show()
print(psutil.cpu_percent())

Berapa banyak item yang dibeli dalam satu transaksi
+--------+------+
|quantity|jumlah|
+--------+------+
|       1|146400|
|       2| 81362|
|      12| 61040|
|       6| 40771|
|       4| 38337|
|       3| 36785|
|      24| 24001|
|      10| 22265|
|       8| 13085|
|       5| 11659|
+--------+------+

100.0


In [ ]:
total_pendapatan_per_produk = spark.sql("SELECT productNo, productName, SUM(Price * Quantity) AS pendapatan FROM sales GROUP BY productNo, productName ORDER BY pendapatan DESC LIMIT 10")
print("Total pendapatan per produk")
total_pendapatan_per_produk.show()
print(psutil.cpu_percent())

Total pendapatan per produk
+---------+--------------------+------------------+
|productNo|         productName|        pendapatan|
+---------+--------------------+------------------+
|    22197|      Popcorn Holder| 583286.5100000016|
|    84077|World War 2 Glide...| 557170.4299999994|
|    23843|Paper Craft Littl...|501359.05000000005|
|   85123A|Cream Hanging Hea...| 451401.6999999961|
|    84879|Assorted Colour B...| 420743.6999999965|
|    21212|Pack Of 72 Retros...|388424.61000000045|
|    23084|  Rabbit Night Light| 327794.2100000003|
|   85099B|Jumbo Bag Red Ret...|         290474.52|
|    22423|Regency Cakestand...| 289288.6000000014|
|    22492|Mini Paint Set Vi...| 284870.9199999996|
+---------+--------------------+------------------+

100.0


In [ ]:
mean_pendapatan_per_transaksi = spark.sql("SELECT AVG(price*quantity) as mean_pendapatan from sales")
print("Rata-rata pendapatan per transaksi")
mean_pendapatan_per_transaksi.show()
print(psutil.cpu_percent())

Rata-rata pendapatan per transaksi
+------------------+
|   mean_pendapatan|
+------------------+
|112.38934326464263|
+------------------+

97.3


In [ ]:
pendapatan_per_hari = spark.sql("SELECT DATE_FORMAT(Date, 'EEEE') as hari, SUM(price*quantity) as pendapatan FROM sales GROUP BY DATE_FORMAT(Date, 'EEEE') ORDER BY pendapatan DESC")
print("Pendapatan per hari")
pendapatan_per_hari.show()
print(psutil.cpu_percent())

Pendapatan per hari
+---------+--------------------+
|     hari|          pendapatan|
+---------+--------------------+
|   Sunday|1.3148277890000954E7|
|   Friday|1.1627657050000742E7|
| Saturday| 1.098026104000066E7|
|   Monday|   9668112.040000133|
| Thursday|   9553636.510000272|
|Wednesday|   5302079.730000019|
+---------+--------------------+

96.7


In [ ]:
pendapatan_harian = spark.sql("SELECT Date, SUM(price*quantity) as pendapatan FROM sales GROUP BY Date ORDER BY pendapatan DESC LIMIT 20")
print("Pendapatan tertinggi pada tanggal tertentu")
pendapatan_harian.show()
print(psutil.cpu_percent())

Pendapatan tertinggi pada tanggal tertentu
+----------+------------------+
|      Date|        pendapatan|
+----------+------------------+
|2019-12-09| 604502.1099999545|
|2019-11-14| 561052.5099999988|
|2019-09-20| 548012.4900000009|
|2019-10-05| 515992.0800000009|
|2019-10-20|469122.96000000305|
|2019-11-10|443115.77000000037|
|2019-08-11| 436995.2399999996|
|2019-08-04| 425920.6899999995|
|2019-05-12| 424704.4700000015|
|2019-09-13|408521.27999999817|
|2019-11-04|405865.65000000177|
|2019-11-03| 400474.4599999992|
|2019-11-09|388492.93000000017|
|2019-03-29|380529.83999999886|
|2019-08-18| 378588.3900000008|
|2019-11-07| 377907.9699999999|
|2019-11-16| 374638.6499999993|
|2019-09-22| 374254.0999999997|
|2019-11-17|369593.02999999904|
|2019-10-27| 367929.0699999986|
+----------+------------------+

99.7


In [ ]:
pendapatan_bulanan = spark.sql("SELECT MONTH(Date) as bulan, SUM(Price*quantity) as pendapatan FROM sales GROUP BY bulan ORDER BY pendapatan DESC")
print("Pendapatan per bulan")
pendapatan_bulanan.show()
print(psutil.cpu_percent())

Pendapatan per bulan
+-----+------------------+
|bulan|        pendapatan|
+-----+------------------+
|   11| 7745257.920000276|
|   10|   6971407.8200005|
|    9| 6542706.300000425|
|   12|  6228326.09999924|
|    8| 4614243.550000117|
|    5| 4530850.360000112|
|    7| 4518347.920000061|
|    6| 4410422.289999971|
|    3|4353308.7800000375|
|    1|3649506.4200000744|
|    4|3416109.2399999956|
|    2| 3299537.560000079|
+-----+------------------+

98.6


In [ ]:
print(psutil.cpu_percent())
selesai_ukur = time.time()

100.0


In [ ]:
total_waktu = selesai_ukur - mulai_ukur
print("Waktu yang dibutuhkan untuk melakukan komputasi: %f detik" % total_waktu )

Waktu yang dibutuhkan untuk melakukan komputasi: 27.690947 detik
